# Clean & preprocess dataset
В этом ноутбуке проводится подготовка и очистка текстов.  
Формируем входной текст  
Удаляем из описаний одежды:  
- артикулы
- размеры
- рекомендации по уходу
- годы
- бренды
- прочий шум


In [1]:
import os
import pandas as pd
import numpy as np
import random
import re

from collections import Counter

## 1. Выгружаем данные

In [ ]:
# выгружаем и сохраняем в один датасет
data_list = []
for name in os.listdir('data'):
    if '.csv'  in name:
        df = pd.read_csv('data/' + name)
        df.drop_duplicates(inplace=True)
        pref = name.split('.')
        df['type'] = pref[0]
        # df['gender'] = pref[0].split('_')[0]
        # df['type'] = pref[0].split('_')[1]
        data_list.append(df)
        
data = pd.concat(data_list, axis = 0)
data.reset_index(drop=True, inplace=True)
data = data[~data.desc.isna()]
data.head()

In [3]:
random_indices = random.sample(range(len(data)), 5)

for idx in random_indices:
    title = data.loc[idx, 'title']
    print(f"Title:\n{title}\n")
    info = data.loc[idx, 'info'] 
    print(f"Info:\n{info}'\n")
    desc = data.loc[idx, 'desc']
    print(f"Description:\n'{desc}'\n")
    print("-"*50)

Title:
Футболка-поло

Info:
: 
Рост модели на фото: 
Параметры модели на фото (ОГ-ОТ-ОБ): 
Страна производства: Турция
Особенности модели: контрастная отделка воротника и манжет
Декоративные элементы: пуговицы
Комплектация: Футболка - поло - 1 шт
Уход за вещами: бережная стирка при 30 градусах
'

Description:
'Стильная мужская футболка-поло турецкого производителя мужской одежды премиум сегмента FILPUCCI с коротким рукавом полуприлегающего кроя. Поло мужское выполнено из натурального дышащего материала, который позволяет одинаково комфортно чувствовать себя как в жаркую погоду, так и в прохладные дни. Базовая модель с коротким рукавом и отложным воротником на пуговицах не сковывает движений и позволяет варьировать образ от более расслабленного до строгого офисного. Эластичный воротник, рукава на трикотажных манжетах и брендированный логотип на груди придают модели изюминку.

Мужская рубашка-поло с коротким рукавом - это стильная альтернатива классической рубашке и спортивной футболке. 

## 2. Формируем входной текст

In [4]:
# уникальные характеристики
info = set()
for s1 in data['info'].dropna():
    l = s1.split('\n')
    for s2 in l:
        if s2:
            info.add(s2.split(':')[0])
info

{'',
 'Вид бретелей',
 'Вид застежки',
 'Вырез горловины',
 'Декоративные элементы',
 'Длина изделия верх',
 'Длина изделия низ',
 'Длина изделия по спинке',
 'Длина по внутреннему шву',
 'Длина юбки/платья',
 'Комплектация',
 'Конструктивные элементы',
 'Материал подкладки',
 'Мембрана',
 'Модель юбки',
 'Опции капюшона',
 'Опции опушки',
 'Особенности белья',
 'Особенности модели',
 'Параметры модели на фото (ОГ-ОТ-ОБ)',
 'Плотность синтетического утеплителя',
 'Покрой',
 'Пол',
 'Размер на модели',
 'Рост модели на фото',
 'Сезон',
 'Состав',
 'Страна производства',
 'Температурный режим',
 'Тип карманов',
 'Тип посадки',
 'Тип рукава',
 'Утеплитель',
 'Уход за вещами',
 'Фактура материала',
 'Характеристика пух/перо (fill power)',
 'Цвет',
 'Ширина низа брючин'}

In [5]:
# выкидываем из характеристик данные по размерам и уходу за вещами
exclude_keywords = [
    'Ширина низа брючин',
    'Длина изделия верх',
    'Длина изделия низ',
    'Длина изделия по спинке',
    'Длина по внутреннему шву',
    'Уход за вещами',
    'Рост модели на фото',
    'Параметры модели на фото (ОГ-ОТ-ОБ)',
    'Размер на модели',
    ''

]

def process_info(text):
    if pd.isna(text):
        return ''
    filtered  = []
    for line in text.strip().split('\n'):
        key, value = line.split(':')[0], line.split(':')[1]
        if key.strip() not in exclude_keywords and value.strip() != '':
            filtered.append(line)
    return '\n'.join(filtered)
    
data['filtered_info'] = data['info'].apply(process_info)

In [6]:
# формируем входной текст: название + характерстики
def create_input(row):
    if row['filtered_info']:
        return f"Сгенерируй описание одежды для карточки товара:\nНаименование товара: {row['title']}\n{row['filtered_info']}"
    return f"Сгенерируй описание одежды для карточки товара:\nНаименование товара: {row['title']}"
data['input'] = data.apply(create_input, axis = 1)
print(data['input'].tolist()[0])

Сгенерируй описание одежды для карточки товара:
Наименование товара: Джеггинсы с высокой посадкой джинсы зауженные скинни
Сезон: демисезон
Утеплитель: без утеплителя
Тип посадки: высокая
Особенности модели: джинсовые брюки на резинке
Декоративные элементы: леггинсы лосины пуш-ап
Комплектация: джеггинсы - 1 шт


## 3. Очистка описаний
Очищаем описания, чтобы убрать шум (артикулы, размеры, уход, бренды) и оставить только полезную информацию для обучения модели.

### 3.1 Информация в скобках

In [10]:
# удаляем второстепенную информацию - все что заключено в скобки
pattern = r'\(([^)]+)\)'

def extract_parentheses_content(text):
    if not isinstance(text, str):
        return None
    
    matches = re.findall(pattern, text)
    return matches if matches else None

def remove_parentheses_content(text):
    if not isinstance(text, str):
        return text
    return re.sub(pattern, '', text)


data['parentheses_content'] = data['desc'].apply(extract_parentheses_content)
data['desc_cleaned_1'] = data['desc'].apply(remove_parentheses_content)

### 3.2 Артикулы

In [11]:
# выкидываем предложения, где встречаются артикулы
pattern = r'(?:[-–—]\s*)?\b(?:артикул|арт\.)\b[\s:\-–—]*[№]?\s*[A-Za-zА-Яа-я0-9/_\-]{2,}(?=[\s.,;!?]|$)'

def replace_art_to_article(text):
    return re.sub(r'\bарт\.\s*', 'артикул ', text, flags=re.IGNORECASE)

def find_descriptions_to_clean(text):
    if not isinstance(text, str):
        return False
    return bool(re.search(pattern, text, re.IGNORECASE))

def split_sentences_with_delimiters(text):
    return re.findall(r'.*?[.!?…\n](?:\s+|$)|.+$', text, flags=re.DOTALL)

def clean_descriptions(text):
    if not isinstance(text, str):
        return text

    text = replace_art_to_article(text)
    sentences = split_sentences_with_delimiters(text)

    cleaned = [s for s in sentences if not re.search(pattern, s, flags=re.IGNORECASE)]

    return ''.join(cleaned).strip()


In [12]:
data['matches'] = data['desc_cleaned_1'].apply(find_descriptions_to_clean)
data['desc_cleaned_2'] = data['desc_cleaned_1'].apply(clean_descriptions)
filtered_data = data[data['matches']]

len(filtered_data)

381

In [13]:
idx = 0
print(f"Pattern:\n{filtered_data.matches.tolist()[idx]}\n")
print(f"Before:\n{filtered_data.desc_cleaned_1.tolist()[idx]}\n")
print(f"After:\n{filtered_data.desc_cleaned_2.tolist()[idx]}\n")

Pattern:
True

Before:
! Артикул брюк в цвете графит 291532688
Базовое утепленное женское худи без капюшона на молнии, изготовлено в России из качественного трикотажа. Внутри свитшота утеплитель - велюр премиального качества . Такой утеплитель не скатывается и не лезет.  Укороченный крой,  широкая цветовая гамма и минималистичный дизайн делают данный джемпер универсальным предметом гардероба. 
Трендовая модель зипки подойдет как для подростков, так и для  взрослых женщин.  

Рекомендации по уходу: деликатная стирка при температуре не более 40°С, отжим не более 600 об/мин.

After:
! Внутри свитшота утеплитель - велюр премиального качества . Такой утеплитель не скатывается и не лезет.  Укороченный крой,  широкая цветовая гамма и минималистичный дизайн делают данный джемпер универсальным предметом гардероба. 
Трендовая модель зипки подойдет как для подростков, так и для  взрослых женщин.  

Рекомендации по уходу: деликатная стирка при температуре не более 40°С, отжим не более 600 об/мин.


### 3.3 Информация по размерам, уходу, тэги

In [14]:
# очищаем текст описаний от информации по размерам, уходу за вещами и прочей шумной информации
def build_general_patterns():
    pattern_size = (
        r"\bразмер(?:а|е|ом|у|ы)?(?:\s+модел[иея]*)?(?:\s+на\s+фото)?\s*[:.]?\s*"
        r"(?:\d{2,5}|XS|S|M|L|XL|XXL|XXXL|XXXXL|5XL|6XL|7XL)"
        r"(?:\s*[-–]\s*\d{2,5})?\b|"
        r"\bразмерами\s*\d{2,5}(?:\s*[-–]\s*\d{2,5})?\b|"
        r"\bна\s+модел[иея]*\s+представлен\s+размер\s+(?:\d{2,5}|XS|S|M|L|XL|XXL|XXXL|XXXXL|5XL|6XL|7XL)\b|"
        r"\bразмер\s+одежды\s+(?:XS|S|M|L|XL|XXL|XXXL|XXXXL|5XL|6XL|7XL)\b|"
        r"\bразмер\s+на\s+модел[иея]*\s*[:.]?\s*(?:\d{2,5}|XS|S|M|L|XL|XXL|XXXL|XXXXL|5XL|6XL|7XL)\b|"
    
        r"\bрост(?:\s+модел[иея]*)?(?:\s+на\s+фото)?\s*[:.]?\s*\d{2,3}(?:[.,]\d)?(?:\s*[-–]\s*\d{2,3}(?:[.,]\d)?)?\s*см?\b|"
        r"\b\w+\s*[-–]\s*\d{2,3}(?:[.,]\d)?\s*см\b|" 
    
        r"(?:ОГ|ОТ|ОБ)(?:\s*[:：]?\s*\d{2,3}){1,3}(?:\s*см)?\b|"
        r"ОГ\s*[-–]\s*ОТ\s*[-–]\s*ОБ\s*[:：]?\s*\d{2,3}(?:[.,]\d)?\s*[-–]\s*\d{2,3}(?:[.,]\d)?\s*[-–]\s*\d{2,3}(?:[.,]\d)?(?:\s*см)?\b|"
        r"параметры\s*(?:модел[иея]*)?\s*[:：]?\s*"
        r"(?:грудь\s*[-–]?\s*\d{2,3}(?:[.,]\d)?\s*см?,?\s*)?"
        r"(?:талия\s*[-–]?\s*\d{2,3}(?:[.,]\d)?\s*см?,?\s*)?"
        r"(?:бёдра|бедра)\s*[-–]?\s*\d{2,3}(?:[.,]\d)?\s*см?\b|"
    
        r"\bвысота\s+талии.*?\d+[,\.]?\d*\s*(?:см|cm)?|"
        r"(?:ширина|длина|обхват|полуобхват|объем(?:\s+груди|\s+талии|\s+бедер)?)(?:\s+\S+){0,3}?\s*[:.]?\s*\d+(?:[-–]\d+)?[,\.]?\d*(?:\s*(?:см|cm))?"
    )


    pattern_care = (
        r"\bстирк[ауеия]{1,2}.*?(?:\d+\s*(?:градус[а-я]*|гр|°C|C))|"
        r"\bможно\s+стирать\s+в\s+машинке|"
        r"\bжидк(ий|им)\s+порошок|"
        r"\bстирка\s+в\s+бережном\s+режиме|бережный\s+режим|мягкий\s+режим|"
        r"\bглажка|глажение|гладить.*?(температур[аы]|утюг|°C|C)|"
        r"\bгладить,\s*не\s+задевая\s+декоративных\s+элементов|"
        r"\bхолодный\s+утюг\s+до\s+\d+\s*°?C|"
        r"\bсушка\s+в\s+машине\s+запрещена|"
        r"\bсушите\s+естественным\s+способом|"
        r"\bсушить|сушка.*?(температуре|в\s+тени|в\s+барабане|на\s+плоскости)|"
        r"\bручная\s+стирка|деликатная\s+стирка|бережная\s+стирка|щадящая\s+стирка|"
        r"\bне\s+гладить|не\s+замачивать|не\s+отбеливать|отбеливание\s+запрещено|"
        r"\bспециальные\s+моющие\s+средства|мягкие\s+моющие\s+средства|"
        r"\bне\s+использовать\s+кондиционер|сильнодействующие\s+моющие\s+средства|"
        r"\bотжим.*?\d+\s*оборот(?:ов)?(?:\s+в\s+минуту)?|"
        r"\bхимчистка\s+запрещена|химическая\s+чистка|"
        r"^рекомендации\s+по\s+уходу[:：]?"
    )

    pattern_meta = (
        r"(?:^|\n)\s*(?:Теги|Нас\s+ищут|Страна\s+производства)[:：]\s*.+|"
        r"(?:^|\n)\s*((?:#\w[\w\dа-яА-ЯёЁ]*)+(?:\s*#\w[\w\dа-яА-ЯёЁ]*)*)"
    )

    return pattern_size, pattern_care, pattern_meta



pattern1_alt, pattern2_alt, pattern3_alt = build_general_patterns()

# прверка наличия паттернов
def find_descriptions_to_clean_alt(text):
    if not isinstance(text, str):
        return False

    if any(sentence.isupper() for sentence in re.split(r'(?<=[.!?])\s+', text)):
        return True

    return any([
        re.search(pattern1_alt, text, re.IGNORECASE),
        re.search(pattern2_alt, text, re.IGNORECASE),
        re.search(pattern3_alt, text, re.IGNORECASE),
    ])

# разбиваем текст на предложения, сохраняя знаки препинания
def split_sentences_with_delimiters(text):
    return re.findall(r'.*?[.!?…\n](?=\s|$)|.+$', text, flags=re.DOTALL)

# функция очистки - выкидывает предложение, если оно целиком в верхнем регистре или содержит инфу о размерах, уходе и прочий шум
def clean_descriptions_alt(text):
    if not isinstance(text, str):
        return text

    sentences = split_sentences_with_delimiters(text)
    cleaned = []
    for sentence in sentences:
        if sentence.isupper():
            continue
        if re.search(pattern1_alt, sentence, re.IGNORECASE):
            continue
        if re.search(pattern2_alt, sentence, re.IGNORECASE):
            continue
        if re.search(pattern3_alt, sentence, re.IGNORECASE):
            continue
        cleaned.append(sentence)

    return ''.join(cleaned).strip()


In [15]:
data['matches'] = data['desc_cleaned_2'].apply(find_descriptions_to_clean_alt)
data['desc_cleaned_3'] = data['desc_cleaned_2'].apply(clean_descriptions_alt)
filtered_data = data[data['matches']]
len(filtered_data)

13490

In [16]:
idx = 13000

print(f"Match: {filtered_data.matches.tolist()[idx]}\n")
print(f"Before:\n{filtered_data.desc_cleaned_2.tolist()[idx]}\n")
print(f"After:\n{filtered_data.desc_cleaned_3.tolist()[idx]}\n")

Match: True

Before:
Эластичный костюм Дэдпула для аниматоров или в качестве костюма на вечеринку или на праздник. Выполнен из эластичного спандекса, который отлично садится на любую фигуру и не сковывает движения. В комплект входит комбинезон, перчатки и маска, МЕЧИ В КОМПЛЕКТ НЕ ВХОДЯТ. Размер 170 подойдет на рост от 160 см, 180 - от 170 см, 190 - от 180 см. Супергерои marvel одежда spiderman марвел дедпул deadpool костюм человека паука, костюм на праздник, одежда на хеллоувин Карнавальный взрослый мужской костюм дэдпула deadpool дедпул костюм на праздник на хеллоуин Костюм или платье на Хэллоуин от лучшего бренда 2024!

After:
Эластичный костюм Дэдпула для аниматоров или в качестве костюма на вечеринку или на праздник. Выполнен из эластичного спандекса, который отлично садится на любую фигуру и не сковывает движения. В комплект входит комбинезон, перчатки и маска, МЕЧИ В КОМПЛЕКТ НЕ ВХОДЯТ. Супергерои marvel одежда spiderman марвел дедпул deadpool костюм человека паука, костюм на пр

### 3.4 Годы

In [17]:
# очищаем описания от годов

# проверка наличия паттерна
def contains_year(text):
    if not isinstance(text, str):
        return False
    return bool(re.search(
        r'\b(?:в|на|до|за|с|из|к)?\s*'
        r'(?:19[7-9]\d|20[0-5]\d)'
        r'(?:[/-–](?:19[7-9]\d|20[0-5]\d))?'
        r'(?:[-–]е|[-–]х)?'
        r'(?:\s*(?:года|год|году))?\b',
        text,
        flags=re.IGNORECASE
    ))
    
# функция очистки от годов
def remove_years_preserve_structure(text):
    if not isinstance(text, str):
        return text

    def clean_paragraph(paragraph):
        # удаляем год в начале предложения
        paragraph = re.sub(
            r'^\s*(?:19[7-9]\d|20[0-5]\d)'
            r'(?:\s*[-–/]\s*(?:19[7-9]\d|20[0-5]\d))?'  
            r'(?:[-–]е|[-–]х)?'
            r'(?:\s*(?:год[ауе]?))?'
            r'\s*[-–—]+\s*',
            '',
            paragraph,
            flags=re.IGNORECASE
        )

        # удаляем года с предлогами
        paragraph = re.sub(
            r'\b(?:в|на|до|за|с|из|к)\s+'
            r'(?:19[7-9]\d|20[0-5]\d)'
            r'(?:[/-–](?:19[7-9]\d|20[0-5]\d))?'
            r'(?:[-–]е|[-–]х)?'
            r'(?:\s*(?:года|год|году))?\b',
            '',
            paragraph,
            flags=re.IGNORECASE
        )

        # удаляем остальные года/ диапазоны (2023/2024)
        paragraph = re.sub(
            r'(?<!осень)(?<!зима)(?<!весна)(?<!лето)'
            r'[\s,:;–-]*'
            r'\b(?:19[7-9]\d|20[0-5]\d)'
            r'(?:[/-–](?:19[7-9]\d|20[0-5]\d))?'
            r'(?:[-–]е|[-–]х)?'
            r'(?:\s*(?:года|год|году))?\b',
            '',
            paragraph,
            flags=re.IGNORECASE
        )

        # очистка пробелов и знаков
        paragraph = re.sub(r'\s{2,}', ' ', paragraph)
        paragraph = re.sub(r'\s+([.,!?])', r'\1', paragraph)
        paragraph = re.sub(r'\s*([–—\-])\s*', r' \1 ', paragraph)
        paragraph = re.sub(r'\s{2,}', ' ', paragraph)

        return paragraph.strip()

    paragraphs = text.split('\n')
    cleaned = [clean_paragraph(p) for p in paragraphs]
    return '\n'.join(cleaned)


In [18]:
data['matches'] = data['desc_cleaned_3'].apply(contains_year)
data['desc_cleaned_4'] = data['desc_cleaned_3'].apply(remove_years_preserve_structure)
filtered_data = data[data['matches']]
len(filtered_data)

11691

In [19]:
idx = -1
print(f"Match: {filtered_data.matches.tolist()[idx]}\n")
print(f"Before:\n{filtered_data.desc_cleaned_3.tolist()[idx]}\n")
print(f"After:\n{filtered_data.desc_cleaned_4.tolist()[idx]}\n")

Match: True

Before:
✦ Рекомендуем ориентироваться на актуальную размерную сетку, расположенную на 8-м фото в карусели фото.

В основе данной куртки улучшенное производство, благодаря этому зимняя мужская модель стоит наравне с настоящим плотным пуховиком. 
Утепленная куртка с капюшоном  обеспечивает превосходную теплоизоляцию в холод не зависимо от погодных условий за счет качественного синтепонового наполнителя.
Дутый фасон этой стильной верхней одежды обеспечит вам тепло в самый суровый мороз. Мембранная вставка в области спины избавит от потоотделения, отстегивающийся капюшон со скрытыми липучками под воротником - стойка защитит лицо от сильных морозов, а манжеты на резинке не дадут пройти холодному воздуху. Качественный подклад также заслуживает внимания: он надежно прошит для длительного срока службы изделия. Оверсайз короткий пуховик имеет водоотталкивающую функцию за счет этого непромокаемая и водонепроницаемая ткань, ветрозащитная планка на молнии надежно защищает от ветра.
Мо

### 3.5 Бренды
Сначала проводим очистку текста для более удобного выявления брендов.  
Ищем упоминания брендов в описаниях (например, «от бренда x», «фирма x» итд), вытаскиваем из них сами названия, а затем заменяет их на обезличенные формулировки вроде «наш бренд», при этом не ломая грамматику и абзацы.

In [20]:
# блок очистки и нормализации текста чтобы потом легче было выявлять названия брендов
import unicodedata

# удаляем диакритики (акценты, тильды итд) - в названиях некотрых брендов встречаются
def remove_accents(text):
    if not isinstance(text, str):
        return text
    # нормализуем в форму NFD, где символы раскладываются на буквы и акценты
    normalized = unicodedata.normalize('NFD', text)
    # удаляем все символы, помеченные как "небуквенные отметки"
    no_accents = ''.join([c for c in normalized if unicodedata.category(c) != 'Mn'])
    return no_accents

# удаляем ненужные символы, кроме: буквы, цифры, пробел, дефис, пунктуация
def remove_unwanted_symbols(text):
    return re.sub(r'[^-A-Za-zА-Яа-яЁё0-9\s,.:;!?\"«»“”%\n]', '', text)

# удаляем символы .:;!?% если они находятся между буквами - часто встречается в названиях брендов
def remove_symbols_inside_words(text):
    return re.sub(r'(?<=[A-Za-z])([.:;!?%]+)(?=[A-Za-z])', '', text)

# переводим латинские слова в верхний регистр, сохраняя структуру абзацев
def capitalize_latin_words_preserve_lines(text):
   
    def capitalize_line(line):
        def capitalize_if_latin(word):
            return word.upper() if re.fullmatch(r'[A-Za-z]+', word) else word
        return ' '.join(capitalize_if_latin(w) for w in line.split())

    return '\n'.join(capitalize_line(line) for line in text.splitlines())


def process_text(text):
    text= remove_accents(text)
    text = remove_unwanted_symbols(text)
    text = remove_symbols_inside_words(text)
    text = capitalize_latin_words_preserve_lines(text)
    return text

data['desc_cleaned_5'] = data['desc_cleaned_4'].apply(process_text)

In [21]:
# ищем фразы упоминания брендов (например, от бренда x, под брендом x, фирмы x)
pattern = re.compile(
    r"""
    (?:от\s+|с\s+|под\s+)?                   
    (?P<prefix>
        [Бб]ренд(?:а|ом)?|
        [Фф]ирм(?:а|ы|ой|е)?|
        [Кк]омпани(?:я|и|ей|ю)?|
        [Мм]агазин(?:а|ом|у)?|
        [Бб]утик(?:а|ом)?
    )
    (?!\s+(Вы|Вас|Вами|Вам|Ваш(?:[еёихемуими])?|Весна(?:-Лето)?|Лето|Осень(?:-Зима)?|Зима|202[34])) 
    \s+
    (?:
        (?P<quote>["'«“])(?P<brand_in_quotes>[^"'\«»“”.,:;!?]+)(?P=quote)  
        |
        (?P<brand>
            (?:
                (?:[A-Z]\.\s*)+(?:[A-Za-z][\w&:/\-.!'?]*)+
                |
                [0-9]+(?:\s+[А-ЯЁ][а-яё0-9\-]*)+
                |
                [0-9A-Za-z][A-Za-z0-9&:/\-.!'?]*(?:\s+[A-Za-z0-9&:/\-.!'?]+)*
                |
                [А-ЯЁ][А-ЯЁа-яё0-9\-]*(?:\s+[А-ЯЁ][А-ЯЁа-яё0-9\-]*)*
            )
        )
    )
    (?<![.\-!?])                                  
    (?=\s+[^\sA-ZА-ЯЁ]|[.,:;!?]|$)                
    """, 
    re.VERBOSE | re.MULTILINE
)

# обрезаем лишнее после бренда, что может захватыватьчя регулярками (например, бренд ТВОЕ Весна-Лето -> бренд ТВОЕ)
def cut_brand_name(brand: str) -> str:
    if not brand:
        return brand
    parts = re.split(r"(\n|\b(?:Вы|Вас|Вами|Вам|Ваш(?:[еёихемуими])?|Весна(?:-Лето)?|Лето|Осень(?:-Зима)?|Зима|202[45])\b)", brand, maxsplit=1)
    return parts[0].strip()

def extract_brand_patterns(text):
    if not isinstance(text, str):
        return None
    matches = pattern.finditer(text)
    result = []
    for m in matches:
        full_phrase = m.group(0).strip()
        cleaned_phrase = cut_brand_name(full_phrase)
        result.append(cleaned_phrase)
    return result or None

# вытаскиваем сами названия брендов (под брендом ZARA -> ZARA)
def extract_brand_names(brand_patterns):
    if not brand_patterns:
        return None
    result = []
    for phrase in brand_patterns:
        name = re.sub(
            r"^(?:от\s+|с\s+|под\s+)?"
            r"(?:[Бб]ренд(?:а|ом)?|[Фф]ирм(?:а|ы|ой|е)?|"
            r"[Кк]омпани(?:я|и|ей|ю)?|[Мм]агазин(?:а|ом|у)?|"
            r"[Бб]утик(?:а|ом)?)\s+",
            "",
            phrase.strip()
        )
        name = name.strip(' "\'«»“”.,:;!?')
        result.append(name)
    return result

# для найденных паттернов выкидываем имя бренда (например, от бренда x -> от бренда)
def clean_brand_patterns(brand_patterns, brand_names):
    if not brand_patterns or not brand_names:
        return None
    return [bp.replace(bn, "").strip() for bp, bn in zip(brand_patterns, brand_names)]
    
# ищем в тексте исходные паттерны и заменяем их на очищенные от названия версии
def replace_brand_patterns(text, brand_patterns, brand_patterns_cleaned):
    if not isinstance(text, str) or not brand_patterns or not brand_patterns_cleaned:
        return text
    for bp, clean_bp in zip(brand_patterns, brand_patterns_cleaned):
        pattern_escaped = r'\s+'.join(map(re.escape, bp.strip().split()))
        text = re.sub(pattern_escaped, clean_bp.strip(), text, flags=re.MULTILINE)
    return text
    
# заменяет отдельно стоящие названия брендов на нейтральные фразы, согласованные по контексту перед брендом
def replace_standalone_brands(text, brand_names):
    if not isinstance(text, str) or not brand_names:
        return text
    escaped_brands = [re.escape(b.strip(' "\'«»“”.,:;!?')) for b in brand_names]
    brand_pattern = r'\b(?:' + '|'.join(escaped_brands) + r')\b'
    def replacer(match):
        start = match.start()
        prev_text = text[max(0, start - 30):start]
        # не трогаем
        if re.search(r'под\s+брендом\s+$', prev_text, flags=re.IGNORECASE):
            return ''
        # от x -> нашего бренда
        elif re.search(r'\bот\s+$', prev_text, flags=re.IGNORECASE):
            return "нашего бренда"
        # с x -> с нашим брендом
        elif re.search(r'\bс\s+$', prev_text, flags=re.IGNORECASE):
            return "с нашим брендом"
        # начало прпдложения: X -> Наш бренд
        elif start == 0 or re.search(r'[.!?]\s*$', text[max(0, start - 2):start]):
            return "Наш бренд"
        # иначе: x -> наш бренд
        else:
            return "наш бренд"
    return re.sub(brand_pattern, replacer, text, flags=re.MULTILINE)

# чистим артефакты после замен
def remove_empty_quotes(text):
    if not isinstance(text, str):
        return text

    paragraphs = text.split('\n')
    cleaned_paragraphs = []

    for para in paragraphs:
        # убираем кавычки из «наш бренд»
        para = re.sub(r'(["\'«»“”])\s*наш бренд\s*\1', '', para, flags=re.IGNORECASE)
        # удаляет пустые кавычки
        para = re.sub(r'\s*["\'«»“”]{2}\s*', ' ', para)
        # удаляет двойные пробелы
        para = re.sub(r'\s{2,}', ' ', para).strip()
        cleaned_paragraphs.append(para)

    return '\n'.join(cleaned_paragraphs)


In [22]:
data['brand_patterns'] = data['desc_cleaned_5'].apply(extract_brand_patterns)
data['brand_names'] = data['brand_patterns'].apply(extract_brand_names)

data['brand_patterns_cleaned'] = data.apply(lambda x: clean_brand_patterns(x['brand_patterns'], x['brand_names']), axis=1)

data['desc_cleaned_6'] = data.apply(lambda x: replace_brand_patterns(x['desc_cleaned_5'], x['brand_patterns'], x['brand_patterns_cleaned']), axis=1)
data['desc_cleaned_7'] = data.apply(lambda x: replace_standalone_brands(x['desc_cleaned_6'], x['brand_names']), axis=1)
data['desc_cleaned_8'] = data['desc_cleaned_7'].apply(remove_empty_quotes)

In [23]:
all_brands = set(data['brand_names'].explode().dropna())
len(all_brands) 

5528

In [24]:
filtered_data = data.dropna(subset=['brand_patterns'])
len(filtered_data)

15074

In [27]:
idx = 0
print(f"Pattern: {filtered_data.brand_patterns.tolist()[idx]}\n")
print(f"Brand: {filtered_data.brand_names.tolist()[idx]}\n")
print(f"Before:\n{filtered_data.desc_cleaned_5.tolist()[idx]}\n")
print(f"After:\n{filtered_data.desc_cleaned_8.tolist()[idx]}\n")

Pattern: ['магазина OBSESSED', 'бренда OBSESSED']

Brand: ['OBSESSED', 'OBSESSED']

Before:
!!! Выбираи размер по ФОТО в карточке товара! Всем привет! На связи команда магазина OBSESSED. Джинсы бананы это универсальные мужские джинсы, которая никогда не выходит из моды. Бананы мужские давно стали популярным элементом мужского гардероба для людеи всех возрастов: подростков, молодых парнеи или взрослых мужчин. Данная модель рассчитана на рост до 185 см. широкии размерныи ряд. Они не красятся! Не растягиваются! Не теряют цвет после стирки! Джинсы мужские, обеспечивают комфорт вне зависимости от сезона. DENIM состоит из 100% высококачественного хлопка, что позволяет ткани дышать в жаркие дни и переносить холод в демисезон. Свободные кроя и являются лучшим вариантом в школу для подростков, на прогулку. Имеют 5 карманов, зауженные к низу. Модные мом придадут уверенности и современности как спортивному, так и повседневному образу. Джинсы стиля боифренд будут еще долгое время актуальны среди м

## 4. Удаляем дубли и сохраняем датасет

In [30]:
data = data.rename(columns = {'desc_cleaned_8': 'final_desc'})

In [33]:
# удаляем строки без описаний и дубликаты
data = data[~data.final_desc.isna()]
data = data.drop_duplicates(subset = ['input', 'final_desc'])
len(data)

79603

In [34]:
# удаляем дупликаты по input, оставляем самое длинное описание
data['len_desc'] = data['final_desc'].apply(lambda x: len(x))
data = data.loc[data.groupby('input')['len_desc'].idxmax()].reset_index(drop = True)
len(data[data.duplicated(subset = 'input', keep = False)])

0

In [36]:
# удаляем дупликаты по описаниям, оставляем самый длинный input
data['len'] = data['input'].apply(lambda x: len(x))
data = data.loc[data.groupby('final_desc')['len'].idxmax()].reset_index(drop = True)
len(data[data.duplicated(subset = 'final_desc', keep = False)])

0

In [56]:
data[['input', 'final_desc', 'type']].to_csv('data_cleaned_short.csv')